# Project Part 1

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)



## Introduction/Background

I'm going to attempt to judge a book by it's title. I'm using the dataset books.csv which was created by kaggle user Soumik by scraping data from the goodreads API (https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks?select=books.csv). Goodreads is a popular website that lists books and allows users to rate and review them. The dataset contains the following features: title, authors, average rating, isbn, language, page numbers, number of ratings, and number of reviews. 

I'm going to develop a model based on this dataset that will take a hypothetical book title as an input, like 'the october rhinocerous' and output a prediction of its rating from 0.00 to 5.00 stars. this could be interpreted as either classification (with each integer amount of stars being a seperate catagory) or regression. As it is more closely aligned with the dataset, i'm going to interpret it as regression: predicting the relationship between the continuous dependent variable rating and the independent variable title.



## Exploratory Data Analysis

_You will now load the dataset and carry out some exploratory data analysis steps to better understand what text data looks like. See the examples from class on 10/. The following links provide some good resources of exploratory analyses of text data with Python._


* https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools
* https://regenerativetoday.com/exploratory-data-analysis-of-text-data-including-visualization-and-sentiment-analysis/
* https://medium.com/swlh/text-summarization-guide-exploratory-data-analysis-on-text-data-4e22ce2dd6ad  
* https://www.kdnuggets.com/2019/05/complete-exploratory-data-analysis-visualization-text-data.html  


In [ ]:
# import all of the python modules/packages you'll need here
import pandas as pd
# ...

books= pd.read_csv('books.csv')
books.head(3)

: 